# Tensorflow Fruits360-VGG16 Patch Demo

>⚠️ **Warning:** This demo assumes that you have access to an on-prem deployment of Dioptra that provides a copy of the Fruits360 dataset and a CUDA-compatible GPU.
> This demo cannot be run on a typical personal computer.

This notebook demonstrates the adversarial patch attack applied on the VGG16 model, as well as adversarial training defenses.

The following two sections cover experiment setup and is similar across all demos.

## Setup: Experiment Name and Fruits360 Dataset

Here we will import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected.

**Important: Users will need to verify or update the following parameters:**

- Ensure that the `USERNAME` parameter is set to your own name.
- Ensure that the `DATASET_DIR` parameter is set to the location of the Fruits360 dataset directory. Currently set to `/nfs/data/Fruits360-Kaggle-2019/fruits-360` as the default location.
- (Optional) Set the `EXPERIMENT_NAME` parameter to your own preferred experiment name.

Other parameters can be modified to alter the RESTful API and MLFlow tracking addresses. 

In [1]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple

# Filter out warning messages
warnings.filterwarnings("ignore")

# Please enter custom username here.
USERNAME = "howard"

# Ensure that the dataset location is properly set here.
DATASET_DIR = "/nfs/data/Fruits360-Kaggle-2019/fruits-360"

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = f"{USERNAME}_fruits360_adversarial_patches"

# Address for connecting the docker container to exposed ports on the host device
HOST_DOCKER_INTERNAL = "host.docker.internal"
# HOST_DOCKER_INTERNAL = "172.17.0.1"

# Testbed API ports
RESTAPI_PORT = "30080"
MLFLOW_TRACKING_PORT = "35000"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = (
    f"http://{HOST_DOCKER_INTERNAL}:{RESTAPI_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{RESTAPI_PORT}"
)

# Override the AI_RESTAPI_URI variable, used to connect to RESTful API service
os.environ["AI_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = (
    f"http://{HOST_DOCKER_INTERNAL}:{MLFLOW_TRACKING_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{MLFLOW_TRACKING_PORT}"
)

# Path to custom task plugins archives
CUSTOM_PLUGINS_EVALUATION_TAR_GZ = Path("custom-plugins-evaluation.tar.gz")

# Override the MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Dataset

The training and testing images in this directory are saved as JPEG files and are organized into the following folder structure:

    fruits360
    ├── Test
    │   ├── Apple Braeburn
    │   ├── Apple Crimson Snow
    │   ├── Apple Golden 1
    │   ├── Apple Golden 2
    │   ├── Apple Golden 3
    │   ├── ...
    │   └── Walnut
    ├── Training
    │   ├── Apple Braeburn
    │   ├── Apple Crimson Snow
    │   ├── Apple Golden 1
    │   ├── Apple Golden 2
    │   ├── Apple Golden 3
    │   ├── ...
    │   └── Walnut

The subfolders under `fruits360/Training/` and `fruits360/Test` are the classification labels for the images in the dataset.
There are 120 labels or subfolders for the training and test sets.
This folder structure is a standardized way to encode the label information and many libraries can make use of it, including the Tensorflow library that we are using for this particular demo.

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `MLproject` file.
To run these entrypoints within the testbed architecture, we need to package those files up into an archive and submit it to the Testbed RESTful API to create a new job.
For convenience, the `Makefile` provides a rule for creating the archive file for this example, just run `make workflows`,

In [2]:
%%bash

# Create the workflows.tar.gz file
make workflows

tar  -czf workflows.tar.gz src/gaussian_augmentation.py src/deploy_patch.py src/init_model.py src/defenses_image_preprocessing.py src/tasks.py src/train.py src/import_keras.py src/spatial_smoothing.py src/generate_patch.py src/jpeg_compression.py src/attacks_patch_updated.py src/fgm.py src/infer.py src/registry_art_updated.py src/estimators_keras_classifiers_updated.py src/data_tensorflow_updated.py MLproject
chmod 644 workflows.tar.gz


/home/hhuang/.conda/envs/tensorflow-mnist-classifier/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the Testbed RESTful API using the HTTP protocol.
We connect using the client below, which uses the environment variable `AI_RESTAPI_URI` to figure out how to connect to the Testbed RESTful API,

In [3]:
restapi_client = utils.SecuringAIClient()

/home/hhuang/.conda/envs/tensorflow-mnist-classifier/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [4]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

{'createdOn': '2020-11-05T09:37:19.652250',
 'experimentId': 11,
 'lastModified': '2020-11-05T09:37:19.652250',
 'name': 'howard_fruits360_adversarial_patches'}

We should also check which queues are available for running our jobs to make sure that the resources that we need are available.
The code below queries the Testbed API and returns a list of active queues.

In [5]:
restapi_client.list_queues()

[{'createdOn': '2020-11-20T17:46:06.756687',
  'queueId': 1,
  'lastModified': '2020-11-20T17:46:06.756687',
  'name': 'tensorflow_cpu'},
 {'createdOn': '2020-11-20T18:00:40.876888',
  'queueId': 2,
  'lastModified': '2020-11-20T18:00:40.876888',
  'name': 'tensorflow_gpu'},
 {'createdOn': '2020-11-20T19:52:36.079781',
  'queueId': 5,
  'lastModified': '2020-11-20T19:52:36.079781',
  'name': 'pytorch_cpu'},
 {'createdOn': '2020-11-20T19:52:43.348460',
  'queueId': 7,
  'lastModified': '2020-11-20T19:52:43.348460',
  'name': 'pytorch_gpu'}]

This example also makes use of the `custom_patch_plugins` custom task plugin package stored locally under the `task-plugins/securingai_custom/custom_patch_plugins` directory.
To register these custom task plugins, we first need to package them up into an archive.
For convenience, the `Makefile` provides a rule for creating the custom task plugins archive file, just run `make custom-plugins`,

In [6]:
%%bash

# Create the workflows.tar.gz file
make custom-plugins

tar -C task-plugins/securingai_custom -czf custom-plugins-evaluation.tar.gz custom_patch_plugins/defenses_image_preprocessing.py custom_patch_plugins/tensorflow.py custom_patch_plugins/import_keras.py custom_patch_plugins/__init__.py custom_patch_plugins/attacks_patch.py custom_patch_plugins/registry_art.py custom_patch_plugins/estimators_keras_classifiers.py custom_patch_plugins/data_tensorflow.py
chmod 644 custom-plugins-evaluation.tar.gz


Now that the custom task plugin package is packaged into an archive file, next we register it by uploading the file to the REST API.
Note that we need to provide the name to use for custom task plugin package, this name must be unique under the custom task plugins namespace.
For a full list of the custom task plugins, use `restapi_client.restapi_client.list_custom_task_plugins()`.

In [7]:
restapi_client.delete_custom_task_plugin(name="custom_patch_plugins")
response_custom_plugins = restapi_client.get_custom_task_plugin(name="custom_patch_plugins")

if response_custom_plugins is None or "Not Found" in response_custom_plugins.get("message", []):
    response_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="custom_patch_plugins",
        custom_plugin_file=CUSTOM_PLUGINS_EVALUATION_TAR_GZ,
    )

response_custom_plugins

/home/hhuang/.conda/envs/tensorflow-mnist-classifier/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'taskPluginName': 'custom_patch_plugins',
 'collection': 'securingai_custom',
 'modules': ['data_tensorflow.py',
  'tensorflow.py',
  'attacks_patch.py',
  'registry_art.py',
  '__init__.py',
  'estimators_keras_classifiers.py',
  'import_keras.py',
  'defenses_image_preprocessing.py']}

If at any point you need to update one or more files within the `evaluation` plugin package, you will need to unregister/delete the custom task plugin first using the REST API.
This can be done as follows,

```python
# Delete the 'evaluation' custom task plugin package
restapi_client.delete_custom_task_plugin(name="custom_patch_plugins")
```

## Adversarial Patches: Baseline Training

Now, we will train our baseline VGG16 model on the Fruits360 dataset. 
We will be submitting our jobs to the `"tensorflow_gpu"` queue.
Once the experiment is finished, we will examine the accuracy results of our model.

In [8]:
response_vgg16_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P batch_size=20",
        f"-P register_model_name={EXPERIMENT_NAME}_vgg16",
        "-P image_size=224,224,3",
        "-P model_architecture=vgg16",
        "-P epochs=30",
        f"-P data_dir_training={DATASET_DIR}/Training",
        f"-P data_dir_testing={DATASET_DIR}/Test",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
)

print("Training job for VGG16 neural network submitted")
print("")
pprint.pprint(response_vgg16_train)

Training job for VGG16 neural network submitted

{'createdOn': '2021-09-23T07:10:28.929177',
 'dependsOn': None,
 'entryPoint': 'train',
 'entryPointKwargs': '-P batch_size=20 -P '
                     'register_model_name=howard_fruits360_adversarial_patches_vgg16 '
                     '-P image_size=224,224,3 -P model_architecture=vgg16 -P '
                     'epochs=30 -P '
                     'data_dir_training=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Training '
                     '-P '
                     'data_dir_testing=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Test',
 'experimentId': 11,
 'jobId': '51c81f39-3bc6-45a3-a56a-1fb0e8482bf0',
 'lastModified': '2021-09-23T07:10:28.929177',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflow/dd3fb6d4ce3a49d2bb86f7d431515494/workflows.tar.gz'}


The following helper functions will recheck the job responses until the job is completed or a run ID is available. 
The run ID is needed to link dependencies between jobs.

In [9]:
def mlflow_run_id_is_not_known(job_response):
    return job_response["mlflowRunId"] is None and job_response["status"] not in [
        "failed",
        "finished",
    ]


def get_run_id(job_response):
    while mlflow_run_id_is_not_known(job_response):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
        
    return job_response


def wait_until_finished(job_response):
    # First make sure job has started.
    job_response = get_run_id(job_response)
    
    # Next re-check job until it has stopped running.
    while (job_response["status"] not in ["failed", "finished"]):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
    
    return job_response    

Now wait for the job to complete before proceeding to next steps.

In [10]:
response_vgg16_train = wait_until_finished(response_vgg16_train)
print("Training job for VGG16 neural network")
pprint.pprint(response_vgg16_train)

Training job for VGG16 neural network
{'createdOn': '2021-09-23T07:10:28.929177',
 'dependsOn': None,
 'entryPoint': 'train',
 'entryPointKwargs': '-P batch_size=20 -P '
                     'register_model_name=howard_fruits360_adversarial_patches_vgg16 '
                     '-P image_size=224,224,3 -P model_architecture=vgg16 -P '
                     'epochs=30 -P '
                     'data_dir_training=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Training '
                     '-P '
                     'data_dir_testing=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Test',
 'experimentId': 11,
 'jobId': '51c81f39-3bc6-45a3-a56a-1fb0e8482bf0',
 'lastModified': '2021-09-23T07:46:19.640608',
 'mlflowRunId': '67b9313763544c41a69a8ff83e1c3231',
 'queueId': 2,
 'status': 'finished',
 'timeout': '1h',
 'workflowUri': 's3://workflow/dd3fb6d4ce3a49d2bb86f7d431515494/workflows.tar.gz'}


## Checking baseline VGG16 job accuracy

Once the job has finished running we can view the results either through the MLflow URI or by accessing the job via MLflow client.
Here we will show the baseline accuracy results from the previous training job.
Please see [Querying the MLFlow Tracking Service](#Querying-the-MLFlow-Tracking-Service) section for more details.

In [11]:
# Helper function for viewing MLflow results.
def get_mlflow_results(job_response):
    mlflow_client = MlflowClient()
    job_response = wait_until_finished(job_response)
    
    if(job_response['status']=="failed"):
        return {}
    
    run = mlflow_client.get_run(job_response["mlflowRunId"])  
    
    while(len(run.data.metrics) == 0):
        time.sleep(1)
        run = mlflow_client.get_run(job_response["mlflowRunId"])
        
    return run


results = get_mlflow_results(response_vgg16_train)
pprint.pprint(results.data.metrics)

{'accuracy': 0.9973983764648438,
 'auc': 0.9998854994773865,
 'loss': 0.009346200119934173,
 'precision': 0.9976862668991089,
 'recall': 0.9971712231636047,
 'restored_epoch': 7.0,
 'stopped_epoch': 12.0,
 'training_time_in_minutes': 27.08974365,
 'val_accuracy': 0.9858291149139404,
 'val_auc': 0.9999104142189026,
 'val_loss': 0.039089418488949965,
 'val_precision': 0.9875176548957825,
 'val_recall': 0.9834258556365967}


## Deploying and Testing Adversarial Patches

Now we will create and apply the adversarial patches over our test set and evaluate the performance of the baseline model on the adversarial patches.
We will also apply the patches over the training set for the adversarial training defense evaluation.

### Patch Generation

The following job will generate the adversarial patches. 
Feel free to adjust the input parameters to see how they impact the effectiveness of the patch attack.

In [12]:
# Create Patches
response_vgg16_patches = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={EXPERIMENT_NAME}_vgg16",
            f"-P model_version=none",
            "-P image_size=224,224,3",
            "-P imagenet_preprocessing=True",
            f"-P data_dir={DATASET_DIR}/Training",
            "-P num_patch_gen_samples=20",
            "-P num_patch=1",
            "-P patch_target=5"
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_vgg16_train["jobId"],
)

print("Patch attack (VGG16 architecture) job submitted")
print("")
pprint.pprint(response_vgg16_patches)
print("")

response_vgg16_patches = get_run_id(response_vgg16_patches)

Patch attack (VGG16 architecture) job submitted

{'createdOn': '2021-09-23T07:46:20.062639',
 'dependsOn': '51c81f39-3bc6-45a3-a56a-1fb0e8482bf0',
 'entryPoint': 'gen_patch',
 'entryPointKwargs': '-P model_name=howard_fruits360_adversarial_patches_vgg16 '
                     '-P model_version=none -P image_size=224,224,3 -P '
                     'imagenet_preprocessing=True -P '
                     'data_dir=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Training '
                     '-P num_patch_gen_samples=20 -P num_patch=1 -P '
                     'patch_target=5',
 'experimentId': 11,
 'jobId': '53cdc2a3-f270-41d7-ad97-d284ef262400',
 'lastModified': '2021-09-23T07:46:20.062639',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/7943a68e20bd4608ad9c95fdbb70fac8/workflows.tar.gz'}



### Deploying and Testing Adversarial Patches

Now we will apply the adversarial patches over our test set and evaluate the performance of the baseline model on the adversarial patches.

In [13]:
# Deploy Patch attack on training set.
response_deploy_vgg16_patches_training = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deploy_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_vgg16_patches['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_vgg16",
            f"-P model_version=none",
            "-P image_size=224,224,3",
            "-P imagenet_preprocessing=True",
            f"-P data_dir={DATASET_DIR}/Training",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_vgg16_patches["jobId"],
)

print("Patch deployment (VGG16 architecture) job submitted")
print("")
pprint.pprint(response_deploy_vgg16_patches_training)
print("")

response_deploy_vgg16_patches_training = get_run_id(response_deploy_vgg16_patches_training)

    
# Deploy Patch attack on test set.
response_deploy_vgg16_patches_testing = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deploy_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_vgg16_patches['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_vgg16",
            f"-P model_version=none",
            "-P image_size=224,224,3",
            "-P imagenet_preprocessing=True",
            f"-P data_dir={DATASET_DIR}/Test",
            "-P patch_deployment_method=corrupt"
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_vgg16_patches["jobId"],
)

print("Patch deployment (VGG16 architecture) job submitted")
print("")
pprint.pprint(response_deploy_vgg16_patches_testing)
print("")

response_deploy_vgg16_patches_testing = get_run_id(response_deploy_vgg16_patches_testing)

Patch deployment (VGG16 architecture) job submitted

{'createdOn': '2021-09-23T07:46:27.236806',
 'dependsOn': '53cdc2a3-f270-41d7-ad97-d284ef262400',
 'entryPoint': 'deploy_patch',
 'entryPointKwargs': '-P run_id=b26a5781c6a146ce86930630043b605a -P '
                     'model_name=howard_fruits360_adversarial_patches_vgg16 -P '
                     'model_version=none -P image_size=224,224,3 -P '
                     'imagenet_preprocessing=True -P '
                     'data_dir=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Training',
 'experimentId': 11,
 'jobId': 'c5943e33-7704-4192-9684-421f9e80cf70',
 'lastModified': '2021-09-23T07:46:27.236806',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/0a12c24ae0004c698f5fed581084d0e1/workflows.tar.gz'}

Patch deployment (VGG16 architecture) job submitted

{'createdOn': '2021-09-23T07:59:24.352161',
 'dependsOn': '53cdc2a3-f270-41d7-ad97-d284ef262400',
 'entryPoint': 'deploy_patc

## Patch Attack Evaluation: Baseline VGG16 Model

Now we will run an inference step to check the patch-attacked dataset with our VGG16-trained model.

In [14]:
# Check patched dataset results   
response_infer_vgg16_patch = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_deploy_vgg16_patches_testing['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_vgg16",
            f"-P model_version=none",
            "-P image_size=224,224,3",
            "-P imagenet_preprocessing=True",
            "-P adv_tar_name=adversarial_patch_dataset.tar.gz",
            "-P adv_data_dir=adv_patch_dataset",
            "-P batch_size=512",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_vgg16_patches_testing["jobId"],
)

print("Patch evaluation (VGG16 architecture) job submitted")
print("")
pprint.pprint(response_infer_vgg16_patch)
print("")

Patch evaluation (VGG16 architecture) job submitted

{'createdOn': '2021-09-23T07:59:31.531198',
 'dependsOn': '5141a991-e001-4215-adec-01022563935d',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=f0b0dd21de8d4f39b9c1740f74a03164 -P '
                     'model_name=howard_fruits360_adversarial_patches_vgg16 -P '
                     'model_version=none -P image_size=224,224,3 -P '
                     'imagenet_preprocessing=True -P '
                     'adv_tar_name=adversarial_patch_dataset.tar.gz -P '
                     'adv_data_dir=adv_patch_dataset -P batch_size=512',
 'experimentId': 11,
 'jobId': 'eef453e0-0a2d-40af-91c9-c14c6fd51538',
 'lastModified': '2021-09-23T07:59:31.531198',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/f875df0a995642099bf5403728c5f6e4/workflows.tar.gz'}



In [15]:
# Wait for the job to finish
response_infer_vgg16_patch = wait_until_finished(response_infer_vgg16_patch)

# Check on the patch evaluation results
results = get_mlflow_results(response_infer_vgg16_patch)
print("Baseline model results on adversarially patched dataset: ")
pprint.pprint(results.data.metrics)

Baseline model results on adversarially patched dataset: 
{'accuracy': 0.020526008680462837,
 'auc': 0.506244421005249,
 'loss': 1675.0153272675304,
 'precision': 0.020528001710772514,
 'recall': 0.020526008680462837}



We can see that the adversarial patch attack causes a noticeable decrease in the model's accuracy scores.

We will now test various defenses against the patch attacked images.

# Defense: Adversarial Training

The next part of the adversarial patch demo focuses on investigating effective defenses against the attack.

### Adversarial Training Defense

We will train a new copy of the VGG16 model on training set that contains adversarial patches.
In doing so, the model learns to ignore the adversarial patches.

In [16]:
response_patches_adv_training = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join(
        [
            f"-P dataset_run_id_testing={response_deploy_vgg16_patches_testing['mlflowRunId']}",
            f"-P dataset_run_id_training={response_deploy_vgg16_patches_training['mlflowRunId']}",
            "-P batch_size=256",
            f"-P register_model_name={EXPERIMENT_NAME}_vgg16",
            "-P image_size=224,224,3",
            "-P imagenet_preprocessing=True",
            "-P epochs=10",
            f"-P data_dir_training={DATASET_DIR}/Training",
            f"-P data_dir_testing={DATASET_DIR}/Test",
            "-P load_dataset_from_mlruns=True",
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_vgg16_patches_training["jobId"],
)

print("Patch adversarial training (VGG16 architecture) job submitted")
print("")
pprint.pprint(response_patches_adv_training)
print("")

response_patches_adv_training = get_run_id(response_patches_adv_training)

Patch adversarial training (VGG16 architecture) job submitted

{'createdOn': '2021-09-23T08:33:10.370580',
 'dependsOn': 'c5943e33-7704-4192-9684-421f9e80cf70',
 'entryPoint': 'train',
 'entryPointKwargs': '-P '
                     'dataset_run_id_testing=f0b0dd21de8d4f39b9c1740f74a03164 '
                     '-P '
                     'dataset_run_id_training=f434549a4826479aa62e16de9a19b0bb '
                     '-P batch_size=256 -P '
                     'register_model_name=howard_fruits360_adversarial_patches_vgg16 '
                     '-P image_size=224,224,3 -P imagenet_preprocessing=True '
                     '-P epochs=10 -P '
                     'data_dir_training=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Training '
                     '-P '
                     'data_dir_testing=/nfs/data/Fruits360-Kaggle-2019/fruits-360/Test '
                     '-P load_dataset_from_mlruns=True',
 'experimentId': 11,
 'jobId': 'ac37d682-a51c-4d6c-9555-a9c655705449',
 'lastModif

In [17]:
response_evaluate_adv_training = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_deploy_vgg16_patches_testing['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_adversarial_patch_vgg16",
            f"-P model_version=none",
            "-P image_size=224,224,3",
            "-P imagenet_preprocessing=True",
            "-P batch_size=256",
            "-P adv_tar_name=adversarial_patch_dataset.tar.gz",
            "-P adv_data_dir=adv_patch_dataset",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_patches_adv_training["jobId"],
)

print("Patch evaluation job submitted")
print("")
pprint.pprint(response_evaluate_adv_training)
print("")

response_patches_adv_training= wait_until_finished(response_evaluate_adv_training)
results = get_mlflow_results(response_patches_adv_training)
print("Adversarial Training Results:")
pprint.pprint(results.data.metrics)

Patch evaluation job submitted

{'createdOn': '2021-09-23T09:38:29.239228',
 'dependsOn': 'ac37d682-a51c-4d6c-9555-a9c655705449',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=f0b0dd21de8d4f39b9c1740f74a03164 -P '
                     'model_name=howard_fruits360_adversarial_patches_adversarial_patch_vgg16 '
                     '-P model_version=none -P image_size=224,224,3 -P '
                     'imagenet_preprocessing=True -P batch_size=256 -P '
                     'adv_tar_name=adversarial_patch_dataset.tar.gz -P '
                     'adv_data_dir=adv_patch_dataset',
 'experimentId': 11,
 'jobId': '88fd081e-375f-49b0-b4ea-9a4885631fc9',
 'lastModified': '2021-09-23T09:38:29.239228',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/0f1feff5139a42cda467f84b41ad696a/workflows.tar.gz'}

Adversarial Training Results:
{'accuracy': 0.1825989931821823,
 'auc': 0.668190598487854,
 'fn': 16890.0,
 'fp': 16649.0,
 'los

## Querying the MLFlow Tracking Service

Currently the lab API can only be used to register experiments and start jobs, so if users wish to extract their results programmatically, they can use the `MlflowClient()` class from the `mlflow` Python package to connect and query their results.
Since we captured the run ids generated by MLFlow, we can easily retrieve the data logged about one of our jobs and inspect the results.
To start the client, we simply need to run,

In [18]:
mlflow_client = MlflowClient()

The client uses the environment variable `MLFLOW_TRACKING_URI` to figure out how to connect to the MLFlow Tracking Service, which we configured near the top of this notebook.
To query the results of one of our runs, we just need to pass the run id to the client's `get_run()` method.
As an example, let's query the run results for the patch attack applied to the VGG16 architecture,

In [19]:
run_adv_patches = mlflow_client.get_run(response_patches_adv_training["mlflowRunId"])

If the request completed successfully, we should now be able to query data collected during the run.
For example, to review the collected metrics, we just use,

In [20]:
pprint.pprint(run_adv_patches.data.metrics)

{'accuracy': 0.1825989931821823,
 'auc': 0.668190598487854,
 'fn': 16890.0,
 'fp': 16649.0,
 'loss': 9.6808679604236,
 'precision': 0.18255020678043365,
 'recall': 0.18041537702083588,
 'tn': 2435703.0,
 'tp': 3718.0}


To review the run's parameters, we use,

In [21]:
pprint.pprint(run_adv_patches.data.params)

{'adv_data_dir': 'adv_patch_dataset',
 'adv_tar_name': 'adversarial_patch_dataset.tar.gz',
 'batch_size': '256',
 'dataset_seed': '390581924',
 'entry_point_seed': '323551244237149393329878256906325458072',
 'image_size': '224,224,3',
 'imagenet_preprocessing': 'True',
 'model_name': 'howard_fruits360_adversarial_patches_adversarial_patch_vgg16',
 'model_version': 'none',
 'run_id': 'f0b0dd21de8d4f39b9c1740f74a03164',
 'seed': '-1',
 'tensorflow_global_seed': '958933669'}


To review the run's tags, we use,

In [22]:
pprint.pprint(run_adv_patches.data.tags)

{'mlflow.project.backend': 'securingai',
 'mlflow.project.entryPoint': 'infer',
 'mlflow.source.name': '/work/tmp3831s6vz',
 'mlflow.source.type': 'PROJECT',
 'mlflow.user': 'securingai',
 'securingai.dependsOn': 'ac37d682-a51c-4d6c-9555-a9c655705449',
 'securingai.jobId': '88fd081e-375f-49b0-b4ea-9a4885631fc9',
 'securingai.queue': 'tensorflow_gpu'}


There are many things you can query using the MLFlow client.
[The MLFlow documentation gives a full overview of the methods that are available](https://www.mlflow.org/docs/latest/python_api/mlflow.tracking.html#mlflow.tracking.MlflowClient).